In [ ]:
import pandas_datareader as wb
import pandas as pd
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt

In [11]:
#뉴스타이틀 크롤링을 위한 모듈 import
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse
from Function import news_title_crawling

In [ ]:
# 코스피지수 크롤링
#약 3년치 데이터를 크롤링해 옴
start = datetime.datetime(2019, 8, 1)
# 현재 날짜까지
end = date.today()

# ^KS11 : 코스피
df_null = wb.DataReader("^KS11","yahoo",start,end)

# 결측치 제거
df = df_null.dropna()

# Close와 Adj Close는 중복되는 columns인것을 확인 함
# Adj Close열을 제거
df.drop(["Adj Close"],axis=1, inplace=True)

# 새로운 칼럼 생성
# (Price : 당일 대비 다음날 주가가 상승했으면 1, 하락했으면 0 표시)
df['Price'] = 0
for i in range(len(df)-1):
    if df['Close'][i] < df['Close'][i+1]:
        df['Price'][i] = 1
    else:
        df['Price'][i] = 0

# columns명을 알기 쉽게 한글로 변경
df.columns = ["최고가", "최저가" , "시작가", "종가", "거래량" , "등락"]

# 파일 저장
df.to_csv('kospi_주가데이터.csv',encoding='utf-8-sig')


In [ ]:
# 등락 값이 "0"인 데이터와 "1"인 데이터를 나눠서 리스트화
# 여기서 리스트화 하는 이유는 다음날 주가가 하락한 경우의 뉴스 타이틀과 상승한 뉴스타이틀을 크롤링해 분류하기 위함
price_data = pd.read_csv('kospi_주가데이터.csv')

#주가가 하락한 경우의 Date 데이터 리스트 date_0
df_0 = price_data[price_data['등락']==0]['Date']
date_0 = []
for i in range(0, len(df_0)):
    date_0.append(str(df_0.tolist()[i])[:10].replace('-', ''))

#주가가 상승한 경우의 Date 데이터 리스트 date_1
df_1 = price_data[price_data['등락']==1]['Date']
date_1 = []
for i in range(0, len(df_1)):
    date_1.append(str(df_1.tolist()[i])[:10].replace('-', ''))

In [ ]:
# 뉴스 타이틀 크롤링한 리스트 병합 후 csv파일로 변환
def news_title_to_csv(site_name:str, date_0, date_1, base_url, news_block, news_title_block):
    result_list = news_title_crawling(date_0, base_url, news_block, news_title_block)
    title_df_0 = pd.DataFrame(result_list, columns=['뉴스제목'])
    title_df_0['주가변동'] = 0

    result_list = news_title_crawling(date_1, base_url, news_block, news_title_block)
    title_df_1 = pd.DataFrame(result_list, columns=['뉴스제목'])
    title_df_1['주가변동'] = 1

    # 크롤링 데이터 합쳐 csv파일로 만들기
    title_df = pd.concat([title_df_0, title_df_1])
    # 중복 데이터 삭제 
    title_df.drop_duplicates(['뉴스제목'])
    # 데이터프레임 저장 
    title_df.to_csv(site_name + '_뉴스타이틀.csv', index=False, encoding='utf-8-sig')

In [ ]:
# news_title_to_csv(site_name:str, date_0, date_1, base_url, news_block, news_title_block)
news_title_to_csv("팍스넷", date_0, date_1,
                base_url='http://www.paxnet.co.kr/news/much?newsSetId=4667&currentPageNo={1}&genDate={0}&objId=N4667',
                news_block='ul.thumb-list li', 
                news_title_block='dl.text > dt')
news_title_to_csv("네이버", date_0, date_1,
                base_url='https://finance.naver.com/news/mainnews.naver?date={0}&page={1}',
                news_block='#contentarea_left > div.mainNewsList > ul > li > dl',
                news_title_block='.articleSubject')
news_title_to_csv("다음", date_0, date_1,
                base_url='https://news.daum.net/breakingnews/economic/stock?page={1}&regDate={0}',
                news_block='ul.list_news2.list_allnews > li',
                news_title_block='.link_txt')
news_title_to_csv("네이트", date_0, date_1,
                base_url='https://news.nate.com/subsection?cate=eco06&mid=n0307&type=c&date={0}&page={1}',
                news_block='div.postSubject > ul > li',
                news_title_block='a')

In [ ]:
#뉴스데이터 하나로 합치기 
paxnet_data = pd.read_csv('팍스넷_뉴스타이틀.csv')
naver_data = pd.read_csv('네이버_뉴스타이틀.csv')
daum_data = pd.read_csv('다음_뉴스타이틀.csv')
nate_data = pd.read_csv('네이트_뉴스타이틀.csv')
all_title = pd.concat([naver_data, paxnet_data, daum_data, nate_data])
all_title.to_csv('news_titles.csv', index=False, encoding='utf-8-sig')